In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
import csv

In [8]:
import pandas as pd

# Read with correct original encoding (common ones: 'utf-8', 'latin1', 'cp1252')
df = pd.read_csv('/content/drive/MyDrive/AOBERT/Data_Cleaned/ethionewsdataset.csv', encoding='latin1')

# Save with UTF-8
df.to_csv('/content/drive/MyDrive/AOBERT/Data_Cleaned/ethionewsdataset11.csv', encoding='utf-8', index=False)

***Preprocess fine tune text-remove special character, URL, hashtag,punctuation, extra space, markup lang***

In [3]:
def preprocess_ft_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s']", '', text)  # Remove special characters except apostrophes(hudhaa)
    text = re.sub(r"http\S+|www.\S+", "", text)  # Remove URLs
    text = re.sub(r"#\w+", "", text)  # Remove hashtags
    text = re.sub(r"[^\w\s]", '', text)  # Remove punctuation and non-alphanumeric characters
    text = re.sub(r"\s+", " ", text)  # Remove extra spaces
    text = re.sub(r"<[^>]+>", "", text)  # Remove markup language
    return text.strip()
with open('/content/drive/MyDrive/AOBERT/Data_Cleaned/ethionewsdataset11.csv', 'r', encoding='latin-1') as infile, \
     open('/content/drive/MyDrive/AOBERT/Data_Cleaned/ethionewsdataset11a.csv', 'w', encoding='utf-8') as outfile:
  reader = csv.reader(infile)
  writer = csv.writer(outfile, delimiter='\t')  # Using tab as delimiter for better readability in text file
  for row in reader:
    row[0]=preprocess_ft_text(row[0])
    writer.writerow(row)

In [4]:
df = pd.read_csv('/content/drive/MyDrive/AOBERT/Data_Cleaned/ethionewsdataset11a.csv', sep='\t', header=None)
print(df.head())

                                                   0      1
0                                               text  label
1  weellistuun beekamtuu ameerikaa biyoonseen wal...      0
2  xiyyaarri imaltootaa raashiyaa erga girrisa si...      0
3  raapparii fi taatoo beekamaan us dmx ganna 50t...      0
4  simbirroonni masqalaa jioota hedduuf erga bada...      0


***Remove Stop words***

In [5]:
def remove_stop_words(text, stop_words_file):
    with open(stop_words_file, 'r') as f:
        stop_words = set([line.strip() for line in f])
    return ' '.join([word for word in text.split() if word not in stop_words])
column_to_clean = 'text' #column containing AO emotional data
with open('/content/drive/MyDrive/AOBERT/Data_Cleaned/ethionewsdataset11a.csv', 'r', encoding='utf-8') as infile, \
     open('/content/drive/MyDrive/AOBERT/Data_Cleaned/ethionewsdataset11ab.csv', 'w', encoding='utf-8') as outfile:
    reader = csv.reader(infile, delimiter='\t')
    writer = csv.writer(outfile, delimiter='\t')
    header = next(reader)  # Read the header row
    writer.writerow(header)  # Write the header to the output file

    for row in reader:
        # Get the index of the column to clean
        try:
            column_index = header.index(column_to_clean)
        except ValueError:
            print(f"Column '{column_to_clean}' not found in header. Skipping row...")
            continue
        # Clean the selected column
        row[column_index] = remove_stop_words(row[column_index], '/content/drive/MyDrive/AOBERT_MODEL/ao_stop_words.txt')
        writer.writerow(row)